In [7]:
import cv2
import numpy as np
import os
import rsa
from google.colab import files
import time

In [8]:
(public_key, private_key) = rsa.newkeys(512)

def rsa_encrypt(message, public_key):
    return rsa.encrypt(message.encode(), public_key)

def rsa_decrypt(encrypted_message, private_key):
    try:
        return rsa.decrypt(encrypted_message, private_key).decode()
    except:
        return "Decryption failed."

In [9]:
def is_valid_image(img):
    if img is None:
        return False
    return True

def is_supported_image_format(extension):
    # List of known supported extensions in OpenCV from the documentation
    supported_extensions = [
        '.bmp', '.dib', '.jpe', '.jp2', '.png', '.webp',
        '.pbm', '.pgm', '.ppm', '.pxm', '.pnm', '.sr', '.ras', '.tiff',
        '.tif', '.exr', '.hdr', '.pic'
    ]
    return extension in supported_extensions

In [10]:
def msg_to_bin(msg):
    if isinstance(msg, str):
        return ''.join([format(ord(i), "08b") for i in msg])
    elif isinstance(msg, (bytes, np.ndarray)):
        return [format(i, "08b") for i in msg]
    elif isinstance(msg, (int, np.uint8)):
        return format(msg, "08b")
    else:
        raise TypeError("Input type not supported")

def hide_data(img, secret_msg):
    n_bytes = img.shape[0] * img.shape[1] * 3 // 8
    print("Maximum Bytes for encoding:", n_bytes)
    secret_msg += '#####'
    if len(secret_msg) > n_bytes:
        raise ValueError("Insufficient bytes, need bigger image or less data!")
    data_index = 0
    bin_secret_msg = msg_to_bin(secret_msg)
    data_len = len(bin_secret_msg)

    for values in img:
        for pixels in values:
            r, g, b = msg_to_bin(pixels)
            if data_index < data_len:
                pixels[0] = int(r[:-1] + bin_secret_msg[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixels[1] = int(g[:-1] + bin_secret_msg[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixels[2] = int(b[:-1] + bin_secret_msg[data_index], 2)
                data_index += 1
            if data_index >= data_len:
                break

    return img

def show_data(img):
    bin_data = ""
    for values in img:
        for pixels in values:
            r, g, b = msg_to_bin(pixels)
            bin_data += r[-1]
            bin_data += g[-1]
            bin_data += b[-1]
    all_bytes = [bin_data[i: i + 8] for i in range(0, len(bin_data), 8)]
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "#####":
            break
    return decoded_data[:-5]

In [11]:
def encode_text():
    print("Upload the image to encode:")

    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Exiting encoding process.")
        return
    img_name = list(uploaded.keys())[0]

    img = cv2.imdecode(np.frombuffer(uploaded[img_name], np.uint8), cv2.IMREAD_COLOR)

    if not is_valid_image(img):
        print("The uploaded file is not a valid image. Please upload a valid image.")
        return

    print("The shape of the image is:", img.shape)

    data = input("Enter data to be encoded: ")
    if len(data) == 0:
        raise ValueError("Data is empty")

    encrypted_message = rsa_encrypt(data, public_key)

    encoded_image = hide_data(img, encrypted_message.hex())

    original_extension = os.path.splitext(img_name)[1].lower()
    if not is_supported_image_format(original_extension):
        output_name = os.path.splitext(img_name)[0] + "_stego.png"
        cv2.imwrite(output_name, encoded_image)
    else:
        output_name = os.path.splitext(img_name)[0] + "_stego" + original_extension
        cv2.imwrite(output_name, encoded_image)

    print(f"Encoded image saved as {output_name}")

def decode_text():
    print("Upload the steganographic image to decode:")

    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Exiting encoding process.")
        return
    img_name = list(uploaded.keys())[0]

    img = cv2.imdecode(np.frombuffer(uploaded[img_name], np.uint8), cv2.IMREAD_COLOR)

    if not is_valid_image(img):
        print("The uploaded file is not a valid image. Please upload a valid image.")
        return

    print("Decoding message...")
    decoded_data = show_data(img)
    print("Decoded message is:", decoded_data)

    decrypted_message = rsa_decrypt(bytes.fromhex(decoded_data), private_key)
    print("Decrypted message:", decrypted_message)

In [12]:
def steganography():
    while True:
        option = input("Image Steganography:\n1. Encode the data\n2. Decode the data\n3. Exit\nSelect the option: ")

        if option == '1':
            print("\nEncoding...")
            encode_text()
        elif option == '2':
            print("\nDecoding...")
            decode_text()
        elif option == '3':
            print("Exiting the program.")
            break
        else:
            print("Invalid option! Please choose 1, 2, or 3.")
        print()

steganography()

Image Steganography:
1. Encode the data
2. Decode the data
3. Exit
Select the option: 1

Encoding...
Upload the image to encode:


Saving turtle.png to turtle.png
The shape of the image is: (240, 240, 3)
Enter data to be encoded: secret message
Maximum Bytes for encoding: 21600
Encoded image saved as turtle_stego.png

Image Steganography:
1. Encode the data
2. Decode the data
3. Exit
Select the option: 2

Decoding...
Upload the steganographic image to decode:


Saving turtle_stego.png to turtle_stego (1).png
Decoding message...
Decoded message is: 2d2195e6fb990478e383f3655f0cea89b38ccb9842d90d2fba974395acb4576cb846321323597289aeafceecbad4a3ce51c867f62920f5171f34a6dbea6ace55
Decrypted message: secret message

Image Steganography:
1. Encode the data
2. Decode the data
3. Exit
Select the option: 1

Encoding...
Upload the image to encode:


Saving slide1.jpg to slide1.jpg
The shape of the image is: (1080, 1080, 3)
Enter data to be encoded: seifgsseb
Maximum Bytes for encoding: 437400
Encoded image saved as slide1_stego.png

Image Steganography:
1. Encode the data
2. Decode the data
3. Exit
Select the option: 2

Decoding...
Upload the steganographic image to decode:


Saving slide1_stego.png to slide1_stego (1).png
Decoding message...
Decoded message is: 8c94fef63026a0376466044a1ab4803502c2138de140bf2fc09f15f89401d9a5c2e3fcbcce16d20d443fe99ac9362afd94692df07891f40566a0b835bb422703
Decrypted message: seifgsseb

Image Steganography:
1. Encode the data
2. Decode the data
3. Exit
Select the option: 3
Exiting the program.
